### Goals 
The goal of this project is to apply some concepts & tools seen in the 3 parts of this course, this
project is organized into 3 parts :<br>
 Part 1 : Building Classical ML projects with respect to basic ML Coding best <br>practices<br>
 Part 2 : Integrate MLFlow to your project<br>
 Part 3 : Integrate ML Interpretability to your project<br>
### DataSet (Finance use case)
DataSet of Home Credit Risk Classification:<br>
https://www.kaggle.com/c/home-credit-default-risk/data<br>
you'll not use all the datasets available on Kaggle, only the main data set :<br>
application_train.csv<br>
application_test.csv<br>
You can also use a part of this dataset (an example is given for this Lab)

In [97]:
import pandas as pd
import numpy as np

# Some sklearn tools for preprocessing and building a pipeline.
# ColumnTransformer was introduced in 0.20 so make sure you have this version
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, classification_report

# Algorithms, from the easiest to the hardest to intepret.r
from xgboost.sklearn import XGBClassifier
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier

In [48]:
app_train = pd.read_csv("application_train.csv",nrows=1000)

In [49]:
app_train.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


In [50]:
#replace every nan of numerical features by the mean of this feature
for column in num_features:    
    app_train[column].fillna((app_train[column].mean()),inplace=True)

In [51]:
app_train[num_features].isnull().sum()

SK_ID_CURR                     0
CNT_CHILDREN                   0
AMT_INCOME_TOTAL               0
AMT_CREDIT                     0
AMT_ANNUITY                    0
AMT_GOODS_PRICE                0
REGION_POPULATION_RELATIVE     0
DAYS_BIRTH                     0
DAYS_EMPLOYED                  0
DAYS_REGISTRATION              0
DAYS_ID_PUBLISH                0
OWN_CAR_AGE                    0
FLAG_MOBIL                     0
FLAG_EMP_PHONE                 0
FLAG_WORK_PHONE                0
FLAG_CONT_MOBILE               0
FLAG_PHONE                     0
FLAG_EMAIL                     0
CNT_FAM_MEMBERS                0
REGION_RATING_CLIENT           0
REGION_RATING_CLIENT_W_CITY    0
HOUR_APPR_PROCESS_START        0
REG_REGION_NOT_LIVE_REGION     0
REG_REGION_NOT_WORK_REGION     0
LIVE_REGION_NOT_WORK_REGION    0
REG_CITY_NOT_LIVE_CITY         0
REG_CITY_NOT_WORK_CITY         0
LIVE_CITY_NOT_WORK_CITY        0
EXT_SOURCE_1                   0
EXT_SOURCE_2                   0
          

In [52]:
app_train[cat_features].dropna(inplace=True)

C:\Users\nana-\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [56]:
len(app_train)

241

In [54]:
app_train.dropna(inplace=True)

In [55]:
app_train[cat_features].isnull().sum()

NAME_CONTRACT_TYPE            0
CODE_GENDER                   0
FLAG_OWN_CAR                  0
FLAG_OWN_REALTY               0
NAME_TYPE_SUITE               0
NAME_INCOME_TYPE              0
NAME_EDUCATION_TYPE           0
NAME_FAMILY_STATUS            0
NAME_HOUSING_TYPE             0
OCCUPATION_TYPE               0
WEEKDAY_APPR_PROCESS_START    0
ORGANIZATION_TYPE             0
FONDKAPREMONT_MODE            0
HOUSETYPE_MODE                0
WALLSMATERIAL_MODE            0
EMERGENCYSTATE_MODE           0
dtype: int64

In [57]:
def missing_values_table(df):
        # Total missing values
        mis_val = df.isnull().sum()
        
        # Percentage of missing values
        mis_val_percent = 100 * df.isnull().sum() / len(df)
        
        # Make a table with the results
        mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
        
        # Rename the columns
        mis_val_table_ren_columns = mis_val_table.rename(
        columns = {0 : 'Missing Values', 1 : '% of Total Values'})
        
        # Sort the table by percentage of missing descending
        mis_val_table_ren_columns = mis_val_table_ren_columns[
            mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1)
        
        # Print some summary information
        print ("Your selected dataframe has " + str(df.shape[1]) + " columns.\n"      
            "There are " + str(mis_val_table_ren_columns.shape[0]) +
              " columns that have missing values.")
        
        # Return the dataframe with missing information
        return mis_val_table_ren_columns

In [58]:
missing_values_table(app_train)

Your selected dataframe has 122 columns.
There are 0 columns that have missing values.


,Missing Values,% of Total Values


In [8]:
# Number of each type of column
app_train.dtypes.value_counts()

float64    65
int64      41
object     16
dtype: int64

In [60]:
# Get X, y
y = app_train["TARGET"]
X = app_train.drop("TARGET", axis=1)

In [61]:
# Some such as default would be binary features, but since
# they have a third class "unknown" we'll process them as non binary categorical
num_features=[]
cat_features=[]
for i in list(zip(X.columns,X.dtypes)):
    if (i[1] != 'object') :
        num_features.append(i[0])
    else : 
        cat_features.append(i[0])

In [62]:
len(num_features),len(cat_features)

(105, 16)

In [116]:
preprocessor = ColumnTransformer([("numerical", "passthrough", num_features),
 ("categorical", OneHotEncoder(sparse=False, handle_unknown='ignore'),
 cat_features)])

In [117]:
# XGBoost
xgb_model = Pipeline([("preprocessor", preprocessor),
 # Add a scale_pos_weight to make it balanced
 ("model", XGBClassifier(scale_pos_weight=(1 - y.mean()), n_jobs=-1))])

# Random Forest
rf_model = Pipeline([("preprocessor", preprocessor),
 ("model", RandomForestClassifier(class_weight="balanced", n_estimators=100))])

#GradientBoostingClassifier
gb_model = Pipeline([("preprocessor", preprocessor),
 ("model", GradientBoostingClassifier())])

In [118]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=.3, random_state=42)

# XGBOOST

In [122]:
gs = GridSearchCV(xgb_model, {"model__max_depth": [5, 10],
 "model__min_child_weight": [5, 10],
 "model__n_estimators": [25]},
 n_jobs=-1, cv=5, scoring="accuracy")
gs.fit(X_train, y_train)

print(gs.best_params_)
print(gs.best_score_)

xgb_model.set_params(**gs.best_params_)
xgb_model.fit(X_train, y_train)

y_pred = xgb_model.predict(X_test)
accuracy_score(y_test, y_pred)

A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.


{'model__max_depth': 5, 'model__min_child_weight': 5, 'model__n_estimators': 25}
0.9166666666666666


0.9041095890410958

# RANDOM FOREST

In [94]:
gs = GridSearchCV(rf_model, {"model__max_depth": [10, 15],
 "model__min_samples_split": [5, 10]},
 n_jobs=-1, cv=5, scoring="accuracy")
gs.fit(X_train, y_train)

print(gs.best_params_)
print(gs.best_score_)

rf_model.set_params(**gs.best_params_)
rf_model.fit(X_train, y_train)

y_pred = rf_model.predict(X_test)
accuracy_score(y_test, y_pred)

{'model__max_depth': 10, 'model__min_samples_split': 5}
0.9166666666666666


C:\Users\nana-\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:813: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


0.9178082191780822

# GRADIENT BOOSTING

In [119]:
gs = GridSearchCV(gb_model, {"model__max_depth": [10, 15],
 "model__min_samples_split": [5, 10]},
 n_jobs=-1, cv=5, scoring="accuracy")
gs.fit(X_train, y_train)

print(gs.best_params_)
print(gs.best_score_)

gb_model.set_params(**gs.best_params_)
gb_model.fit(X_train, y_train)

y_pred = gb_model.predict(X_test)
accuracy_score(y_test, y_pred)

The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.


{'model__max_depth': 10, 'model__min_samples_split': 10}
0.9047619047619048


0.8767123287671232

import eli5
eli5.show_weights(rf_model.named_steps["model"])
#That gives us the weights associated to each feature, that can be seen as the contribution of each feature into
#predicting that the class will be y=1 (the client will subscribe after the campaign).
#The names for each features aren't really helping though, we can pass a list of column names to eli5 but
#we'll need to do a little gymnastics first to extract names from our preprocessor in the pipeline (since we've
#generated new features on the fly with the one hot encoder)

preprocessor = rf_model.named_steps["preprocessor"]
ohe_categories = preprocessor.named_transformers_["categorical"].categories_
new_ohe_features = [f"{col}__{val}" for col, vals in zip(cat_features, ohe_categories) for val in vals]
all_features = num_features + new_ohe_features
eli5.show_weights(rf_model.named_steps["model"], feature_names=all_features)

import shap
# Need to load JS vis in the notebook
shap.initjs() 

explainer = shap.TreeExplainer(xgb_model.named_steps["model"])

observations = xgb_model.named_steps["preprocessor"].transform(X_train.sample(1000, random_state=42))
shap_values = explainer.shap_values(observations)

i = 0
shap.force_plot(explainer.expected_value, shap_values[i],
 features=observations[i], feature_names=all_features)

shap.force_plot(explainer.expected_value, shap_values,
 features=observations, feature_names=all_features)

shap.summary_plot(shap_values, features=observations, feature_names=all_features)